# Dataset contains headlines and categories for over 400k news articles

Using different model to classify the headlines.Models are provided in Python by the scikit-learn library.

### Naive Bayes model

In [1]:
import pandas as pd 

# the Naive Bayes model
from sklearn.naive_bayes import MultinomialNB
# function to split the data for cross-validation
from sklearn.model_selection import train_test_split
# function for transforming documents into counts and remove stopwords.
from sklearn.feature_extraction.text import TfidfVectorizer
# function for encoding categories
from sklearn.preprocessing import LabelEncoder
# function for give recall,precesion,f1-score,support
from sklearn.metrics import classification_report
# function for using chi squared test
from sklearn.feature_selection import chi2,SelectKBest
# function to perform cross validation
from sklearn.model_selection import cross_val_score
#function to use KFold 
from sklearn.model_selection import KFold


In [2]:
data = pd.read_csv("news.csv")

data.head()

,ID,TITLE,URL,PUBLISHER,CATEGORY,STORY,HOSTNAME,TIMESTAMP
0,1,"Fed official says weak data caused by weather,...",http://www.latimes.com/business/money/la-fi-mo...,Los Angeles Times,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.latimes.com,1394470370698
1,2,Fed's Charles Plosser sees high bar for change...,http://www.livemint.com/Politics/H2EvwJSK2VE6O...,Livemint,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.livemint.com,1394470371207
2,3,US open: Stocks fall after Fed official hints ...,http://www.ifamagazine.com/news/us-open-stocks...,IFA Magazine,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.ifamagazine.com,1394470371550
3,4,"Fed risks falling 'behind the curve', Charles ...",http://www.ifamagazine.com/news/fed-risks-fall...,IFA Magazine,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.ifamagazine.com,1394470371793
4,5,Fed's Plosser: Nasty Weather Has Curbed Job Gr...,http://www.moneynews.com/Economy/federal-reser...,Moneynews,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.moneynews.com,1394470372027



Removing punctuation and lowercase everything. This will give us a smaller set of words, which will decrease the size of our model, and ensure that words are treated the same even if they occur capitalized at the beginning of the headline or lowercase in the middle.



In [3]:

tf = TfidfVectorizer(stop_words = 'english', 
                     token_pattern=u'(?ui)\\b\\w*[a-z]+\\w*\\b')

x = tf.fit_transform(data['TITLE'])


In [26]:
#print(x[1:100])
tf.vocabulary_

txt_fitted = tf.fit(data['TITLE'])

idf = tf.idf_

print(dict(zip(txt_fitted.get_feature_names(), idf)))

{'0017a43b2370': 13.260608178178188, '00am': 13.260608178178188, '00msk': 13.260608178178188, '00p': 13.260608178178188, '00pm': 11.874313817058297, '00z': 11.756530781401914, '01am': 12.567460997618243, '01pc': 13.260608178178188, '020d': 13.260608178178188, '025bn': 13.260608178178188, '02am': 12.567460997618243, '02c1e10a03f0_story': 13.260608178178188, '03am': 12.855143070070023, '03bn': 12.855143070070023, '04am': 12.855143070070023, '04b': 12.855143070070023, '04pm': 13.260608178178188, '05am': 12.855143070070023, '05b': 13.260608178178188, '05bn': 12.855143070070023, '06am': 13.260608178178188, '06b': 13.260608178178188, '06pm': 12.855143070070023, '06z': 12.855143070070023, '072b': 13.260608178178188, '07am': 13.260608178178188, '07b': 13.260608178178188, '07pm': 13.260608178178188, '07z': 13.260608178178188, '08am': 13.260608178178188, '08pc': 12.855143070070023, '09am': 13.260608178178188, '09b': 12.855143070070023, '09pm': 13.260608178178188, '09t12': 13.260608178178188, '0a

In [4]:
#using encoder to encode the label into interger
encoder = LabelEncoder()
y = encoder.fit_transform(data['CATEGORY'])


# split into train and test sets in 80:20 ratio
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

#print(type(x.shape),set(y))
# take a look at the shape of each of these
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)


(337935, 51642)
(337935,)
(84484, 51642)
(84484,)


In [5]:
#Making classifer
nb = MultinomialNB()
nb.fit(x_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [9]:
#calucation of the accuracy of the model
nb.score(x_test, y_test)

0.9284361535912126

In [10]:
#taking account of the perfomance of the model using classification report

x_test_pred = nb.predict(x_test)
print(classification_report(y_test, x_test_pred,target_names=encoder.classes_))

              precision    recall  f1-score   support

           b       0.90      0.92      0.91     23159
           e       0.96      0.97      0.96     30430
           m       0.97      0.86      0.91      9219
           t       0.91      0.91      0.91     21676

   micro avg       0.93      0.93      0.93     84484
   macro avg       0.93      0.91      0.92     84484
weighted avg       0.93      0.93      0.93     84484



In [40]:
#Finding most co-related term in each sample feature using chi squared test
ch2 = SelectKBest(chi2,k=1).fit_transform(x, y)


In [41]:
#fucntion to reverse the vectorized word back to its orignal form

def make_reverse_vocabulary(vectorizer):
    revvoc = {}

    vocab = vectorizer.vocabulary_
    for w in vocab:
        #i = vocab[w]
        print(w)
        #revvoc[i] = w

    #return revvoc

make_reverse_vocabulary(tf)

fed
official
says
weak
data
caused
weather
slow
taper
s
charles
plosser
sees
high
bar
change
pace
tapering
open
stocks
fall
hints
accelerated
risks
falling
curve
nasty
curbed
job
growth
accelerate
expects
unemployment
end
jobs
month
hit
president
ecb
unlikely
sterilisation
smp
purchases
traders
sterilization
eu
half
baked
bank
union
work
europe
reaches
crunch
point
banking
focus
stronger
euro
drowns
message
rates
low
aims
deal
tackling
failing
banks
forex
pound
drops
lows
noyer
strong
creates
unwarranted
economic
pressure
week
ahead
march
resolution
transparency
ukraine
member
kinds
tools
anxieties
wane
bunds
treasuries
spain
debt
rallies
economy
bad
loan
triggers
key
feature
test
announcement
sources
china
trade
deficit
structural
worries
main
issues
things
need
know
morning
happy
strength
update
eurozone
sovereign
exposure
hits
new
reveal
hurdles
zone
market
eur
usd
retreats
year
highs
seen
refile
requires
tougher
definition
tests
opening
bell
central
contain
threats
prices
threat
ri

apples
user
experience
hinted
jailbroken
xcode
upgrade
direct
links
images
gallery
ibettercharge
charge
home
discovered
code
chances
appear
slim
previously
vehicle
enable
hdr
5s
include
hyped
breaks
visuals
administrators
applaud
connected
strategy
interface
accessibility
pod2g
faq
extra
unleashes
owners
facetime
handy
immediately
warning
sports
festival
perfect
installed
older
dials
meld
impressed
pandora
diss
maps
product
cycles
impress
wow
criticism
experts
raised
spat
spills
remote
browse
purchased
movies
airplay
environments
bonjour
blocked
hide
unwanted
channels
channel
firmware
11d169b
welcome
icon
clutter
cleaning
paralympic
easy
ability
hides
unwatched
kiwi
tops
nz
charts
fyi
manually
restart
hiding
added
revamped
concert
publishsosimply
publishing
magazine
replica
card
ohio
steady
rhode
island
gallon
aaa
honolulu
outlook
savannah
peoria
expert
refinery
maintenance
cause
spike
continuing
connecticut
montana
massachusetts
jump
climbing
harrisburg
national
rochester
bellingham
d

takeaways
jefferies
cautions
fy14
climbs
f4q
current
income
row
cautious
spdr
etf
powershares
qqq
wary
disturbing
budget
premarket
broadly
firmer
steadies
rupee
fx
steadier
footing
boj
shaken
disappointment
unable
derail
drift
gain
jitters
evening
lamphier
legs
debating
live5news
charleston
trim
grinds
fmc
yen
health
carney
select
rbs
england
overhauls
regulatory
independent
scotland
quizzed
rigging
libor
spotlight
started
qe
housing
extraordinary
shake
probes
suspended
alarm
appoint
deputy
governor
gbp
loose
destroys
tapes
meetings
extremely
boe
investigations
lender
resort
scots
independence
grilled
trims
remains
keeping
shades
nixon
ftse100
overheating
fixing
knowing
manipulation
lawmaker
handling
encouraging
ftse
bullish
explain
members
election
unreasonable
headquarters
conduct
authority
investigating
distinct
possibility
divided
output
stalls
voters
remarks
tighten
persist
ruthless
allegation
iditarod
mile
viable
relentlessly
maintain
fisher
hunting
wrongdoing
slipped
testified
s

piz
marshmallow
cheat
adulthood
cult
flair
infographic
echolls
silly
scott
pierce
cs
interviews
downfall
leather
feisty
wcw
diehard
satisfied
fandom
flashback
checking
tch
resurrecting
faithful
suspense
clued
gang
reprises
necessarily
roth
cornet
neptune
dirt
bros
originally
topics
explored
reviving
dvd
recording
bedroom
wows
nights
goth
gothic
glam
tortured
haircut
cosies
mandy
grading
backer
newbie
homecoming
tasering
luscious
arch
campbell
nancy
drew
hat
peeing
oscars
stuck
orbit
musicals
fandango
momentous
rave
marshmallows
thriller
wisecracking
recaptures
rogan
screenings
rooney
mara
pan
racial
tiger
lily
forthcoming
fairytale
prequel
jackman
wright
whitest
chick
coudt
whitewashing
lilly
opposite
garrett
hedlund
mcdill
similar
genuine
whines
wages
previous
owe
carla
rodriguez
mama
rundown
divorces
rycroft
disastrous
disasterous
girlfirend
jillian
unsatisfying
cheer
schools
dedicated
cdl
stirs
bullying
glad
hater
bass
forgives
feuding
tabloids
reigned
spectacle
speculate
uploads
ar

elusive
reiniers
wasserman
schultz
davie
shhh
desperation
mounts
webinar
pulse
extending
penalty
hall
sylvia
garcia
hispanics
mesa
column
bumpy
fla
hardship
broward
millennials
dooming
swedish
forefront
wellpoint
penalties
columbus
dispatch
misleadingly
prizes
lowcountry
hertz
equipment
rental
separated
renting
htz
brks
mksi
atec
naples
actiev
amzn
orcl
annex
seperated
separate
structure
traded
sirius
xm
buyback
hounews
spins
6c
separation
pistorius
gorbachev
yemen
aden
wal
mart
sampler
primary
kagan
happiest
herc
divestment
leasing
avis
cube
weakened
bulgaria
registrations
vw
amended
acea
imf
tucking
handle
broadbent
nemat
shafik
setter
appointments
ishares
msci
ewu
governors
appointment
stamps
radical
ceiling
guard
osborne
departure
institution
posen
values
haldane
cv
appointed
iou
imbalance
illusions
neutral
removed
attributed
sina
weibo
apology
spearhead
tue
12am
township
buick
enclave
cadillac
xts
traverse
gmc
czar
string
18m
airbag
boyer
additional
dealing
redoubles
assure
intens

...

clairton
wtrf
omnibus
criminalizing
landowners
koch
inputs
underpinned
retreated
folling
usd1300
usd102
str
wagers
liven
69b
1qfy15
funk
fax
expections
disquieting
intension
outgrow
trex
fq1
fyq1
maxim
rybolovlev
oligarch
costliest
62c
penthouse
fertilizer
billonaire
potash
potentate
toh
elena
rybolovleva
5bln
alimony
7billion
francs
48bn
us4
2014russian
2014corrected
whipsaw
disadvantaged
scrapbook
knit
apk
scrapbooks
nocturnal
clarion
nightsnake
nightsnakes
herpetologists
mulcahy
ucs
jamestown
verde
ultralight
laplet
ergonomic
absurdly
featherweights
conceding
trig
i7
ultrathin
fledged
climategate
waned
unstirred
zane
croak
supercell
fundamentalists
iea
unengaged
wdp
fiddles
wmn
soaking
forbids
eww
retaliates
emphatic
goodell
matsui
whipping
rollback
logjam
gandel
aeroflex
oru
romano
poore
terrace
slayings
honestly
kneecap
volte
tangible
beachhead
gs4
ibmedge
swath
pester
nosier
bewildering
badger
liddick
blaster
vrizzmo
ndreams
eutugxiqhry
skydieving
ception
frail
gaunt
johnathan
re

intercontinentalexchange
perecnt
433k
xhb
veiga
rothwell
preholiday
doomsdayers
tenneco
unimpeded
terrify
harrods
confiscated
wave3
mcconnell
ridesharing
wto
steelworkers
ustr
itn
komodo
slink
exhibited
500e
bardot
consisting
zoomable
mosaics
rack
bings
kairos
jellycar
solitaire
scdca
reders
escalated
hatchette
zola
informational
interlock
2500s
3500s
324m
collusion
325m
meeellion
406m
g4
gtat
scuttlebutt
circumpolar
delsanto
1am
seti
milkyway
reportpublish
astrobiologists
ets
extraterrestrials
shostak
rebadged
jims
counseled
oseary
pissing
doosh
disassembled
afte
helming
exited
aaaarrrrrghhh
wight
inhaling
inhales
chipmunk
squeaky
wormhole
kornel
mundruczo
nouvelle
2014dogs
hungarian
feher
isten
gump
paterson
uma
mush
spaghetti
joblo
dawdles
shakurâ
œf
youâ
unflinching
medici
descendant
cheatsheet
kanyestein
bedsheet
conceive
pilates
ancien
régime
garavani
chiacago
kanya
2014kim
decoy
scottie
pippen
hirst
cadet
aishwarya
amfar
supermodels
strut
magpies
5million
refaeli
beauties
ducky


collars
metta
notoriously
carpetbagger
63rd
gobber
nettles
picassos
kravings
athena
kourteny
kontroller
scooby
calmness
shadoe
wove
ellington
soothed
feuded
casem
dedications
deejay
rermember
yer
prescott
middletons
rik
mayall
hotdog
abstaining
prediabetic
prediabetes
lipid
beeb
cdph
sproutbreaks
teammates
parrsboro
marlow
attleboro
rowlesburg
valparaiso
joliet
relaywood
zeeland
133k
moshannon
dillsburg
tomato
toujeo
mayflies
algae
fertilizers
wmb
midstream
acmp
unitholders
bocomm
zloty
eassurance
normalisation
15t10
15t17
07z
ratestom
normalization
esteemed
recognitions
authorizations
paise
georgie
geyer
mcx
medvedev
belarusian
connivance
oettinger
petro
unconstructive
snips
ukrainy
delude
illarionov
vilnius
prepay
prepays
jihad
prepayments
unsc
lukashenko
minsk
cutoff
m2m
cobra
telematics
fabrizio
bensch
renounce
siptu
rebase
gnp
boro
oilfields
ruehl
shales
usd4
micex
storages
95b
khead
andrii
deshchytsia
tudor
pickering
wpz
2014airbus
launchers
twtc
dara
biosciences
herda
affymax
lv

spectrometers
oco2
encountered
vibrations
munching
caterpillars
beastly
specced
rs37800
maximized
minimalism
spiel
gfx
s8
s10
txt
cramming
chained
overcharging
fraudulently
crammed
broadsides
coeur
alene
hydroplane
appraisals
bilked
tucked
jacking
totalling
aptenodytes
forsteri
depts
itis
inverter
unblemished
bigfoot
unsurprising
sasquatch
yetis
hairs
himalayas
obliterated
chilean
oceanic
forage
garco
feltonville
torresdale
inhospitable
gliese
832c
battlestar
galactica
uninhabitable
cabal
driveless
spaniard
crawly
emoto
furtling
klitzman
ithacavoice
221back
morstan
abbington
broil
cunt
bau
07pm
whisperer
sarchie
oliva
predictability
trots
exorcisms
demonic
édgar
heebie
jeebies
hoariest
demonologist
howler
chesterton
deliever
flannery
jazmine
backlashes
annihilated
kiddnapping
ranchero
kink
lt
winsome
untugged
dkeira
mazursky
notables
eras
unmarried
farceur
admiral
countercultural
tonto
voluntarytreatment
tenenbaum
deviant
grouchiest
unmistakably
babysits
kicky
jaeden
lieberher
racetrac

roving
nasas
offearth
lunokhod
orbiters
repositions
reposition
seawater
acidification
0t
duran
prinze
teamkiefer
unprofessionalism
indytoday
bloodfeud
trailerculture
rhys
pabbie
whaddya
bilbo
baggins
thorin
gandalf
legolas
bowman
smaug
elves
birthed
unto
sdcc
annabelle
glorous
jester
elvish
nerdgasmic
tp
koin
cosplayers
cosplaying
zombiewalk
hawtness
ra
ghul
expos
olutely
leveled
rozanski
louvre
stranding
kosovo
madagascar
degrading
nuttymadam
bowls
batkid
leukemia
beysus
divorcè
stylists
igf
winnipeggers
surfboardt
sweethearts
stepbrother
rubbed
vacuous
micromanages
10x8
milf
smooched
tinted
bomby
istants
neglects
snogged
wifey
controling
neglecting
klips
ziering
snarknado
snuffing
saws
churl
classless
kabc7
richt
embiid
wotv
murin
dorfamn
hy
hitchhiker
rooker
racoon
xd
assortment
faris
braiding
170m
guradians
kwok
hyperdrive
rubdown
comrie
lizzie
fantasizes
daydreams
daydreamin
daydreaming
snacktime
summery
skywriting
confections
jazzed
jennny
handjobs
zeal
rilo
kiley
kimono
phobias


In [42]:
#using the k-fold technique to increase the accuracy of the model

cv = KFold(n_splits = 10,shuffle = True, random_state=0)
result = cross_val_score(nb, x, y, cv=cv)
print(result.mean())

0.9272688990572527


#### <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

### Logistic regression model

#### >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

In [17]:
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier

In [18]:
clf = OneVsRestClassifier(LogisticRegression())

# Fit the classifier to the training data
clf.fit(x_train, y_train)


# Print the accuracy
print("Accuracy: {}".format(clf.score(x_test, y_test)))

/home/aman/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Accuracy: 0.9419771791108376


In [19]:
x_test_reg = clf.predict(x_test)

print(classification_report(y_test, x_test_reg,target_names=encoder.classes_))

              precision    recall  f1-score   support

           b       0.92      0.92      0.92     23047
           e       0.96      0.98      0.97     30287
           m       0.96      0.91      0.93      9262
           t       0.93      0.92      0.93     21888

   micro avg       0.94      0.94      0.94     84484
   macro avg       0.94      0.93      0.94     84484
weighted avg       0.94      0.94      0.94     84484



In [20]:
#result = cross_val_score(clf, x, y, cv=cv)
#print(result.mean())

#### <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

### Random Forest

#### >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

In [21]:
from sklearn.ensemble import RandomForestClassifier



In [22]:
clf_rand = RandomForestClassifier()
# Fit the classifier to the training data
clf_rand.fit(x_train, y_train)

# Print the accuracy
print("Accuracy: {}".format(clf_rand.score(x_test, y_test)))

/home/aman/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Accuracy: 0.9276431040196961


In [23]:
x_test_rand = clf_rand.predict(x_test)

print(classification_report(y_test, x_test_rand,target_names=encoder.classes_))

              precision    recall  f1-score   support

           b       0.90      0.92      0.91     23047
           e       0.94      0.97      0.96     30287
           m       0.94      0.88      0.91      9262
           t       0.94      0.90      0.92     21888

   micro avg       0.93      0.93      0.93     84484
   macro avg       0.93      0.92      0.92     84484
weighted avg       0.93      0.93      0.93     84484



#### <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

### Decision Trees

#### >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

In [24]:
from sklearn import tree


In [25]:
clf_tree = tree.DecisionTreeClassifier()

clf_tree.fit(x_train, y_train)

# Print the accuracy
print("Accuracy: {}".format(clf_tree.score(x_test, y_test)))

Accuracy: 0.914587377491596


In [26]:
x_test_tree = clf_tree.predict(x_test)

print(classification_report(y_test, x_test_tree,target_names=encoder.classes_))

              precision    recall  f1-score   support

           b       0.89      0.90      0.89     23047
           e       0.94      0.95      0.95     30287
           m       0.90      0.88      0.89      9262
           t       0.91      0.89      0.90     21888

   micro avg       0.91      0.91      0.91     84484
   macro avg       0.91      0.91      0.91     84484
weighted avg       0.91      0.91      0.91     84484

